In [93]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

In [94]:
crsp=pd.read_csv('crsp.csv')
crsp.head()

,PERMNO,date,SHRCD,EXCHCD,DLRET,PRC,RET,SHROUT
0,10000,19851231,NaN,NaN,NaN,NaN,NaN,NaN
1,10000,19860131,10.0,3.0,NaN,-4.3750,C,3680.0
2,10000,19860228,10.0,3.0,NaN,-3.2500,-0.257143,3680.0
3,10000,19860331,10.0,3.0,NaN,-4.4375,0.365385,3680.0
4,10000,19860430,10.0,3.0,NaN,-4.0000,-0.098592,3793.0


In [95]:
share_codes=[10,11]
exch_codes=[1,2,3]
crsp=crsp[crsp.SHRCD.isin(share_codes) & crsp.EXCHCD.isin(exch_codes)]
crsp.head()

,PERMNO,date,SHRCD,EXCHCD,DLRET,PRC,RET,SHROUT
1,10000,19860131,10.0,3.0,NaN,-4.37500,C,3680.0
2,10000,19860228,10.0,3.0,NaN,-3.25000,-0.257143,3680.0
3,10000,19860331,10.0,3.0,NaN,-4.43750,0.365385,3680.0
4,10000,19860430,10.0,3.0,NaN,-4.00000,-0.098592,3793.0
5,10000,19860530,10.0,3.0,NaN,-3.10938,-0.222656,3793.0


In [96]:
crsp['DLRET']=pd.to_numeric(crsp.DLRET,errors='coerce')
crsp['RET']=pd.to_numeric(crsp.RET,errors='coerce')
crsp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3544881 entries, 1 to 4513909
Data columns (total 8 columns):
PERMNO    int64
date      int64
SHRCD     float64
EXCHCD    float64
DLRET     float64
PRC       float64
RET       float64
SHROUT    float64
dtypes: float64(6), int64(2)
memory usage: 243.4 MB


In [97]:
crsp['cumret']=crsp.RET
crsp.loc[crsp.RET.isnull(),'cumret']=crsp.DLRET
crsp.loc[crsp.RET.notnull() & crsp.DLRET.notnull(),'cumret']=(1+crsp.DLRET)*(1+crsp.RET)-1
crsp.head()

,PERMNO,date,SHRCD,EXCHCD,DLRET,PRC,RET,SHROUT,cumret
1,10000,19860131,10.0,3.0,NaN,-4.37500,NaN,3680.0,NaN
2,10000,19860228,10.0,3.0,NaN,-3.25000,-0.257143,3680.0,-0.257143
3,10000,19860331,10.0,3.0,NaN,-4.43750,0.365385,3680.0,0.365385
4,10000,19860430,10.0,3.0,NaN,-4.00000,-0.098592,3793.0,-0.098592
5,10000,19860530,10.0,3.0,NaN,-3.10938,-0.222656,3793.0,-0.222656


In [98]:
crsp['date']=pd.to_datetime(crsp.date,format="%Y%m%d")
crsp=crsp.sort_values(by=['PERMNO','date'])
crsp.head()

,PERMNO,date,SHRCD,EXCHCD,DLRET,PRC,RET,SHROUT,cumret
1,10000,1986-01-31,10.0,3.0,NaN,-4.37500,NaN,3680.0,NaN
2,10000,1986-02-28,10.0,3.0,NaN,-3.25000,-0.257143,3680.0,-0.257143
3,10000,1986-03-31,10.0,3.0,NaN,-4.43750,0.365385,3680.0,0.365385
4,10000,1986-04-30,10.0,3.0,NaN,-4.00000,-0.098592,3793.0,-0.098592
5,10000,1986-05-30,10.0,3.0,NaN,-3.10938,-0.222656,3793.0,-0.222656


In [99]:
crsp['mktcap']=abs(crsp.PRC)*crsp.SHROUT
#crsp['mktcap_lag']=crsp.mktcap.shift(1)
crsp['mktcap_lag']=crsp.groupby('PERMNO')['mktcap'].shift(1)
crsp.head()

,PERMNO,date,SHRCD,EXCHCD,DLRET,PRC,RET,SHROUT,cumret,mktcap,mktcap_lag
1,10000,1986-01-31,10.0,3.0,NaN,-4.37500,NaN,3680.0,NaN,16100.00000,NaN
2,10000,1986-02-28,10.0,3.0,NaN,-3.25000,-0.257143,3680.0,-0.257143,11960.00000,16100.0
3,10000,1986-03-31,10.0,3.0,NaN,-4.43750,0.365385,3680.0,0.365385,16330.00000,11960.0
4,10000,1986-04-30,10.0,3.0,NaN,-4.00000,-0.098592,3793.0,-0.098592,15172.00000,16330.0
5,10000,1986-05-30,10.0,3.0,NaN,-3.10938,-0.222656,3793.0,-0.222656,11793.87834,15172.0


In [100]:
#crsp['rank_ret']=crsp.cumret.rolling(12,min_periods=8).sum()-crsp.cumret.rolling(1).sum()
crsp['rank_ret']=crsp.groupby('PERMNO')['cumret'].transform(lambda x: x.rolling(12,min_periods=8).sum()-x.rolling(1).sum())
crsp.head()

,PERMNO,date,SHRCD,EXCHCD,DLRET,PRC,RET,SHROUT,cumret,mktcap,mktcap_lag,rank_ret
1,10000,1986-01-31,10.0,3.0,NaN,-4.37500,NaN,3680.0,NaN,16100.00000,NaN,NaN
2,10000,1986-02-28,10.0,3.0,NaN,-3.25000,-0.257143,3680.0,-0.257143,11960.00000,16100.0,NaN
3,10000,1986-03-31,10.0,3.0,NaN,-4.43750,0.365385,3680.0,0.365385,16330.00000,11960.0,NaN
4,10000,1986-04-30,10.0,3.0,NaN,-4.00000,-0.098592,3793.0,-0.098592,15172.00000,16330.0,NaN
5,10000,1986-05-30,10.0,3.0,NaN,-3.10938,-0.222656,3793.0,-0.222656,11793.87834,15172.0,NaN


In [101]:
crsp['year']=crsp.date.dt.year
crsp['month']=crsp.date.dt.month
crsp_final=crsp.loc[:,['date','month','year','PERMNO','EXCHCD','mktcap_lag','cumret','rank_ret']]
crsp_final.head()

,date,month,year,PERMNO,EXCHCD,mktcap_lag,cumret,rank_ret
1,1986-01-31,1,1986,10000,3.0,NaN,NaN,NaN
2,1986-02-28,2,1986,10000,3.0,16100.0,-0.257143,NaN
3,1986-03-31,3,1986,10000,3.0,11960.0,0.365385,NaN
4,1986-04-30,4,1986,10000,3.0,16330.0,-0.098592,NaN
5,1986-05-30,5,1986,10000,3.0,15172.0,-0.222656,NaN


In [102]:
crsp_final['DT_deciles']=crsp_final.groupby('date')['rank_ret'].transform(lambda x: pd.qcut(x,10,labels=False,duplicates='drop'))
crsp_final.head()

,date,month,year,PERMNO,EXCHCD,mktcap_lag,cumret,rank_ret,DT_deciles
1,1986-01-31,1,1986,10000,3.0,NaN,NaN,NaN,NaN
2,1986-02-28,2,1986,10000,3.0,16100.0,-0.257143,NaN,NaN
3,1986-03-31,3,1986,10000,3.0,11960.0,0.365385,NaN,NaN
4,1986-04-30,4,1986,10000,3.0,16330.0,-0.098592,NaN,NaN
5,1986-05-30,5,1986,10000,3.0,15172.0,-0.222656,NaN,NaN


In [103]:
total_cap=crsp_final.groupby(['date','DT_deciles'])['mktcap_lag'].sum().reset_index()
total_cap.columns=['date','DT_deciles','total_cap_lag']
crsp_final=crsp_final.merge(total_cap,on=['date','DT_deciles'])
crsp_final.head()

,date,month,year,PERMNO,EXCHCD,mktcap_lag,cumret,rank_ret,DT_deciles,total_cap_lag
0,1987-01-30,1,1987,10000,3.0,1981.56609,-0.212121,-1.531149,0.0,1.681080e+07
1,1987-01-30,1,1987,10005,3.0,433.68750,0.666667,-0.727506,0.0,1.681080e+07
2,1987-01-30,1,1987,10007,3.0,23415.00000,0.321429,-0.738364,0.0,1.681080e+07
3,1987-01-30,1,1987,10013,3.0,857.43750,0.000000,-1.244714,0.0,1.681080e+07
4,1987-01-30,1,1987,10018,3.0,2705.12500,0.000000,-1.098940,0.0,1.681080e+07


In [104]:
crsp_final['weight']=crsp_final.mktcap_lag/crsp_final.total_cap_lag
crsp_final['DT_ret']=crsp_final.weight*crsp_final.cumret
crsp_final.head()

,date,month,year,PERMNO,EXCHCD,mktcap_lag,cumret,rank_ret,DT_deciles,total_cap_lag,weight,DT_ret
0,1987-01-30,1,1987,10000,3.0,1981.56609,-0.212121,-1.531149,0.0,1.681080e+07,0.000118,-0.000025
1,1987-01-30,1,1987,10005,3.0,433.68750,0.666667,-0.727506,0.0,1.681080e+07,0.000026,0.000017
2,1987-01-30,1,1987,10007,3.0,23415.00000,0.321429,-0.738364,0.0,1.681080e+07,0.001393,0.000448
3,1987-01-30,1,1987,10013,3.0,857.43750,0.000000,-1.244714,0.0,1.681080e+07,0.000051,0.000000
4,1987-01-30,1,1987,10018,3.0,2705.12500,0.000000,-1.098940,0.0,1.681080e+07,0.000161,0.000000


In [105]:
crsp_final['DT_deciles']=crsp_final.DT_deciles+1
pd.unique(crsp_final.DT_deciles)

array([ 1.,  7.,  6.,  5.,  4.,  8.,  9.,  3.,  2., 10.])

In [106]:
DT_replicate=crsp_final.groupby(['month','year','DT_deciles'])['DT_ret'].sum()
DT_replicate=DT_replicate.reset_index()
DT_replicate=DT_replicate.sort_values(by=['year','month','DT_deciles'])
DT_replicate.head()

,month,year,DT_deciles,DT_ret
10120,12,1926,1.0,0.094171
10121,12,1926,2.0,0.030302
10122,12,1926,3.0,0.036827
10123,12,1926,4.0,0.017593
10124,12,1926,5.0,0.012447


In [107]:
DT_orig=pd.read_excel('daniel.xlsx',names=['date','DT_deciles','DT_orig_ret','cum','no.'])
DT_orig.head()

,date,DT_deciles,DT_orig_ret,cum,no.
0,19270131,1,-0.03362,7325.5,46
1,19270131,2,-0.04584,11500.9,47
2,19270131,3,0.02755,23650.0,47
3,19270131,4,-0.00319,42549.1,47
4,19270131,5,-0.00294,46375.2,47


In [108]:
DT_orig['date']=pd.to_datetime(DT_orig.date,format="%Y%m%d")
DT_orig['month']=DT_orig.date.dt.month
DT_orig['year']=DT_orig.date.dt.year
DT_orig=DT_orig.loc[:,['month','year','DT_deciles','DT_orig_ret']]
DT_orig.head()

,month,year,DT_deciles,DT_orig_ret
0,1,1927,1,-0.03362
1,1,1927,2,-0.04584
2,1,1927,3,0.02755
3,1,1927,4,-0.00319
4,1,1927,5,-0.00294


In [109]:
corr_data=DT_replicate.merge(DT_orig,on=['month','year','DT_deciles'])
corr_data.head()

,month,year,DT_deciles,DT_ret,DT_orig_ret
0,1,1927,1.0,-0.042326,-0.03362
1,1,1927,2.0,-0.030936,-0.04584
2,1,1927,3.0,0.016902,0.02755
3,1,1927,4.0,-0.005052,-0.00319
4,1,1927,5.0,0.002560,-0.00294


In [110]:
corr_data.DT_ret.corr(corr_data.DT_orig_ret)

0.9475731079306899